# IMPORT MODULES

In [1]:
import sys

sys.path.append('..')

In [2]:
from modules.model import DLMA
from modules.dataset import DataGenerator
import numpy as np
import glob

Using TensorFlow backend.


# IMPORT MODEL

In [3]:
model = DLMA(time_steps=3, classes_num=2)
model.compile(optimizer='rmsprop', loss='kullback_leibler_divergence', metrics=['accuracy'])

W0819 11:46:41.815756 140608137897792 deprecation_wrapper.py:119] From /home/grodri/anaconda3/envs/keras_apnea/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0819 11:46:41.831229 140608137897792 deprecation_wrapper.py:119] From /home/grodri/anaconda3/envs/keras_apnea/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0819 11:46:41.833716 140608137897792 deprecation_wrapper.py:119] From /home/grodri/anaconda3/envs/keras_apnea/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0819 11:46:41.889010 140608137897792 deprecation_wrapper.py:119] From /home/grodri/anaconda3/envs/keras_apnea/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.place

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 3, 128)       0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 3, 1024)      132096      input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 3, 1024)      4096        dense_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 3, 1024)      0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
dropout_1 

# IMPORT DATASET

In [4]:
train_path = '../data/arrays/train/'
dev_path   = '../data/arrays/dev/'
test_path  = '../data/arrays/test/'

partition  = dict()

In [5]:
train_db = glob.glob(train_path + '*.npy')
dev_db   = glob.glob(dev_path + '*.npy')
test_db  = glob.glob(test_path + '*.npy')

In [6]:
# Take only the file name, not the full path
partition['train'] = [train_db[i].split('/')[-1] for i in range(len(train_db))]
partition['dev']   = [dev_db[i].split('/')[-1] for i in range(len(dev_db))]
partition['test']  = [test_db[i].split('/')[-1] for i in range(len(test_db))]

In [7]:
train_generator = DataGenerator(
    partition['train'],
    db_part='train',
    dim=(3, 128),
    batch_size=32,
    n_classes=2,
    shuffle=True
)

dev_generator   = DataGenerator(
    partition['dev'],
    db_part='dev',
    dim=(3, 128),
    batch_size=32,
    n_classes=2,
    shuffle=True
)

test_generator  = DataGenerator(
    partition['test'],
    db_part='test',
    dim=(3, 128),
    batch_size=32,
    n_classes=2,
    shuffle=True
)

# TRAIN

In [8]:
model.fit_generator(
    generator=train_generator,
    validation_data=dev_generator,
    workers=8,
    epochs=100
)

W0819 11:47:06.791498 140608137897792 deprecation.py:323] From /home/grodri/anaconda3/envs/keras_apnea/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
528/528 [==============================] - 26s 49ms/step - loss: 0.6519 - acc: 0.5697 - val_loss: 1.8140 - val_acc: 0.4996
Epoch 2/100
528/528 [==============================] - 24s 45ms/step - loss: 0.6370 - acc: 0.5846 - val_loss: 1.2446 - val_acc: 0.4996
Epoch 3/100
528/528 [==============================] - 22s 43ms/step - loss: 0.6325 - acc: 0.5888 - val_loss: 2.6718 - val_acc: 0.5004
Epoch 4/100
528/528 [==============================] - 22s 42ms/step - loss: 0.6296 - acc: 0.5910 - val_loss: 3.1908 - val_acc: 0.5004
Epoch 5/100
528/528 [==============================] - 22s 43ms/step - loss: 0.6281 - acc: 0.5915 - val_loss: 1.0118 - val_acc: 0.5004
Epoch 6/100
528/528 [==============================] - 23s 43ms/step - loss: 0.6270 - acc: 0.5950 - val_loss: 1.1044 - val_acc: 0.5000
Epoch 7/100
528/528 [==============================] - 23s 43ms/step - loss: 0.6254 - acc: 0.5967 - val_loss: 3.6788 - val_acc: 0.5000
Epoch 8/100
528/528 [==============================] - 

528/528 [==============================] - 23s 44ms/step - loss: 0.6198 - acc: 0.6034 - val_loss: 2.7546 - val_acc: 0.4996
Epoch 62/100
528/528 [==============================] - 24s 45ms/step - loss: 0.6179 - acc: 0.6015 - val_loss: 2.8009 - val_acc: 0.5004
Epoch 63/100
528/528 [==============================] - 24s 45ms/step - loss: 0.6199 - acc: 0.6040 - val_loss: 2.7636 - val_acc: 0.4992
Epoch 64/100
528/528 [==============================] - 24s 45ms/step - loss: 0.6187 - acc: 0.6029 - val_loss: 2.7967 - val_acc: 0.5004
Epoch 65/100
528/528 [==============================] - 24s 45ms/step - loss: 0.6197 - acc: 0.6045 - val_loss: 2.8102 - val_acc: 0.5004
Epoch 66/100
528/528 [==============================] - 24s 45ms/step - loss: 0.6182 - acc: 0.6027 - val_loss: 2.8293 - val_acc: 0.4996
Epoch 67/100
528/528 [==============================] - 24s 45ms/step - loss: 0.6191 - acc: 0.6039 - val_loss: 2.8163 - val_acc: 0.4992
Epoch 68/100
528/528 [==============================] - 24s 4

In [ ]:
model.save_weights('weights_100_epochs.h5')

# TEST MODEL

In [ ]:
model.load_weights('weights_100_epochs.h5')

In [ ]:
model.evaluate_generator(generator=test_generator, verbose=1)

In [ ]:
pred = model.predict_generator(test_generator, verbose=1)
pred = np.argmax(pred, axis=1)